In [7]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
%matplotlib inline

In [8]:
import math
import collections      #import collections class to create a deque data structure
previous_frames = collections.deque(maxlen = 10)  

def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    (assuming your grayscaled image is called 'gray')
    you should call plt.imshow(gray, cmap='gray')"""
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Or use BGR2GRAY if you read an image with cv2.imread()
    # return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

def draw_lines_challenge(img, lines, color=[255, 0, 0], thickness=10):
    xsize = img.shape[1]
    ysize = img.shape[0]
    left_lines = []         #create a new list containing lines on the left
    right_lines = []        #create a new list containing lines on the right
    for line in lines:
        for x1,y1,x2,y2 in line:
            if (y2-y1)/(x2-x1) > math.tan(math.pi/9):             #if the angle is greater than 20 degree, then it belongs to one of the right lines
                right_lines.append([x1,y1,x2,y2])
            elif (y2-y1)/(x2-x1) < -math.tan(math.pi/9):       #otherwise the line should be on the left side
                left_lines.append([x1,y1,x2,y2])
            else:
                continue              
    #convert from list to np.array whose dimension should be (number of lines,4)
    left = np.array(left_lines)
    right = np.array(right_lines)
    
    if left.shape[0]<=1 or right.shape[0]<=1:
        coefs = np.array([[], []])
    else:
        #using np.polyfit to obtain the coefficients of the linear line on each side
        left_coefs = np.polyfit(left[:,(0,2)].ravel(), left[:,(1,3)].ravel(), 1)
        right_coefs = np.polyfit(right[:,(0,2)].ravel(), right[:,(1,3)].ravel(), 1)
        coefs = np.array([left_coefs, right_coefs]) 
        #get coordinate(x,y) of four points, two of which is on each sides
        left_y1 = ysize
        left_x1 = int(round((left_y1 - left_coefs[1]) / left_coefs[0]))
        left_y2 = int(0.65 * ysize)
        left_x2 = int(round((left_y2 - left_coefs[1]) / left_coefs[0]))
    
        right_y1 = ysize
        right_x1 = int(round((right_y1 - right_coefs[1]) / right_coefs[0]))
        right_y2 = int(0.65 * ysize)
        right_x2 = int(round((right_y2 - right_coefs[1]) / right_coefs[0]))
        #draw two lines on the image
        cv2.line(img, (left_x1,left_y1), (left_x2,left_y2), color, thickness)
        cv2.line(img, (right_x1,right_y1), (right_x2,right_y2), color, thickness)
         
    return coefs

def hough_lines_challenge(img, rho, theta, threshold, min_line_len, max_line_gap):

    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    coefs = draw_lines_challenge(line_img, lines)
    
    return line_img, coefs


def weighted_img(img, initial_img, α=0.8, β=1, λ=0.):

    return cv2.addWeighted(initial_img, α, img, β, λ)

In [9]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [10]:
def process_image_challenge(image):
    # NOTE: The output you return should be a color image (3 channel) for processing video below
    # TODO: put your pipeline here,
    # you should return the final output (image where lines are drawn on lanes)
    
    #Define the parameters which will be used later
    low_threshold = 30
    high_threshold = 150
    kernel_size = 5
    rho = 2
    theta = np.pi/180
    threshold =15
    min_line_len = 40
    max_line_gap = 20
    color=[255, 0, 0]
    thickness=10

    xsize = image.shape[1]
    ysize = image.shape[0]
    vertices = np.array([[(0,ysize),(430, 0.60*ysize), (540, 0.60*ysize), (xsize,ysize)]], dtype=np.int32)
    
    gray = grayscale(image)
    blur_gray = gaussian_blur(gray, kernel_size)
    edges = canny(blur_gray, low_threshold, high_threshold)
    masked_edges = region_of_interest(edges, vertices)

    line_image, coefs = hough_lines_challenge(masked_edges, rho, theta, threshold, min_line_len, max_line_gap)
    if coefs.size==0:      #if the array of either left side or right side is empty
        #averaging the coefficients collected from previous 10 frames
        previous_coefs = np.array(previous_frames)
        left_avg = np.mean(previous_coefs[:,0,:], axis=0)
        right_avg = np.mean(previous_coefs[:,1,:], axis=0)
        
        #get coordinate(x,y) of four points, two of which is on each sides
        left_y1 = ysize
        left_x1 = int(round((left_y1 - left_avg[1]) / left_avg[0]))
        left_y2 = int(0.65 * ysize)
        left_x2 = int(round((left_y2 - left_avg[1]) / left_avg[0]))
    
        right_y1 = ysize
        right_x1 = int(round((right_y1 - right_avg[1]) / right_avg[0]))
        right_y2 = int(0.65 * ysize)
        right_x2 = int(round((right_y2 - right_avg[1]) / right_avg[0]))
        #draw two lines on the image
        cv2.line(line_image, (left_x1,left_y1), (left_x2,left_y2), color, thickness)
        cv2.line(line_image, (right_x1,right_y1), (right_x2,right_y2), color, thickness)
    else:   #if the we got lines from both sides, then we append the coefficients to the deque
        previous_frames.append(coefs)
        
    result = weighted_img(line_image, image, α=0.8, β=1., λ=0.)      
    return result

In [11]:
challenge_output = 'challenge1.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
##clip3 = VideoFileClip('test_videos/challenge.mp4').subclip(0,5)
clip3 = VideoFileClip('test_videos/challenge.mp4')
challenge_clip = clip3.fl_image(process_image_challenge)
%time challenge_clip.write_videofile(challenge_output, audio=False)

[MoviePy] >>>> Building video challenge1.mp4
[MoviePy] Writing video challenge1.mp4


100%|██████████| 251/251 [00:19<00:00, 14.61it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: challenge1.mp4 

CPU times: user 8.64 s, sys: 1.89 s, total: 10.5 s
Wall time: 21.6 s


In [12]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(challenge_output))